## Top solutions
As most of you will already know, there is a [public spreadsheet](https://docs.google.com/spreadsheets/u/0/d/e/2PACX-1vQ7RUqwrtwRD2EJbgMRrccAHkwUQZgFe2fsROCR1WV5LA1naxL0pU2grjQpcWC2HU3chdGwIOUpeuoK/pubhtml?pli=1#gid=0) where many teams have shared their best scores on each of the 400 tasks. This provides a useful comparison tool and I am very grateful for the shared data.

There is an understanding on kaggle that medal scoring notebooks should not be shared in the last part of the competition. At least, this used to be the case, but I have not participated for years and am not sure if it is written down anywhere or still applies. However, to this end I have taken 2 precautions.
1. I have chosen tasks that illustrate a strategy, but fall behind top scores reported in the link above. The examples from yesterday and today do trim hundreds of characters off the existing public solutions but fall short of top ones. The task in this notebook takes a current public solution **438 bytes down to 108**, but the reported top solution is only 65! I have no clue how they chip it in from there.
2. I am **deliberately not merging** these notebooks with public solution datasets to shoot them to the front the `top public score` filter to get copies and likes. That way, by the letter of the law at least, they are not medal-capable solutions.

In this notebook I will explore the idea of identifying a pattern in a recursive function and then repeating it.  

Haha, it would be easy to beat the top solution to this task if it **really was** 65!

## Chop and copy
Many of the simpler tasks with short solutions involve some form of pattern copy. I will use `task 343` as an example of this technique.

In [ ]:
import sys
sys.path.append("/kaggle/input/google-code-golf-2025/code_golf_utils")
from code_golf_utils import *

In [ ]:
task_num = 343
show_examples(load_examples(task_num)['train'])
show_examples(load_examples(task_num)['arc-gen'][:3])

## Copy the pattern
The only tricky part of this task is finding the pattern length `q`. Once that is done, it is simple to produce the output.  
  
An **extra trick** to note below is that the solution size is hard-coded as 15. This **works for all** examples in this task. Noticing this kind of data uniformity in the tasks often saves a lot of valuable bytes.

In [ ]:
example=load_examples(task_num)["train"][0];g=example["input"]
q=3 # This q value of is the all-important pattern size. Once calculated, the solution is simple.
[(r[:q]*15)[:15]for r in g]

In [ ]:
q=3
example["output"]=[(r[:q]*15)[:15]for r in g]
show_examples([example])

## Searching for q
To state the obvious, this will not work for examples with a different pattern size. It does already pass more than 40% of the tasks as shown below.

In [ ]:
%%writefile task.py
p=lambda g:[(r[:3]*15)[:15]for r in g]

In [ ]:
verify_program(task_num, load_examples(task_num))

## First remove input padding
In order to find the pattern length we will recursively try lengths of `1,2,3,...` until one gives the required pattern.  

At each step in the recursion, we will construct an output and then compare compare that with the available input. When doing this comparison, there is no hard-coded amount of available input that works for all examples. I have tried. The only option is to use all the input available. 

So we first need to remove the padding to the right of the input. The best method I can find to do this involves transposing the input with `zip` and then `filter`ing for rows that have `any` colour in them. There are 2 methods of filtering shown below. The first is more self-axplanatory but the second is shorter.

In [ ]:
[r for r in zip(*g)if any(r)]

In [ ]:
[*filter(any,[*zip(*g)])]

## Recursion rocks
We can now use recursion, starting at `q=1` and test our constructed output with the filtered input. Once these match, we return a result for the lambda function p.

In [ ]:
p=lambda g,q=1:p(g,q+1)if((k:=[*filter(any,[*zip(*g)])])[:q]*9)[:len(k)]!=k else[(r[:q]*15)[:15]for r in g]

In [ ]:
example=load_examples(task_num)["train"][2];
solution={"input":example["input"],"output":p(example["input"])}
show_examples([example,solution])

## Final algorithm
We now test this algorithm against the full set to see if it passes.

In [ ]:
%%writefile task.py
p=lambda g,q=1:p(g,q+1)if((k:=[*filter(any,[*zip(*g)])])[:q]*9)[:len(k)]!=k else[(r[:q]*15)[:15]for r in g]

In [ ]:
verify_program(task_num, load_examples(task_num))

## Version edits below, based on input from others in the comments
Some interesting improvements have been shared in the comments, bringing us closer to the 65. Thanks for the ideas.  

Firstly, [jacekwl](https://www.kaggle.com/jacekwl) pointed out that `filter` takes an iterable as second argument, So the function can be simplified to:

In [ ]:
p=lambda g,q=1:p(g,q+1)if((k:=[*filter(any,zip(*g))])[:q]*9)[:len(k)]!=k else[(r[:q]*15)[:15]for r in g]

Then [bizy-coder](https://www.kaggle.com/benzydney) noticed that all pattern lengths are a factor of 8 or 6. So we can simplify by finding the right boolean expression to substitute into the function below:

In [ ]:
p=lambda g:[(r[:(6,8)[False]]*15)[:15]for r in g]

[Nazar_Mykola](https://www.kaggle.com/havefan1234)havefan1234 has found one option, reducing the byte count to 89

In [ ]:
p=lambda g:[(r[:(8,6)[((k:=[*filter(any,zip(*g))])[:6]*9)[:len(k)]==k]]*5)[:15]for r in g]

I then found my own variation, bringing it to 78 bytes.

In [ ]:
p=lambda g:[(r[:(6,8)[(t:=[*zip(*g)])[:4]in[t[4:8],t[8:12]]]]*15)[:15]for r in g]

Zipping the zip back, instead of using a direct list comprehension gets it to 78 bytes.

In [ ]:
p=lambda g:[*zip(*((t:=[*zip(*g)])[:(6,8)[t[:4]in[t[4:8],t[8:12]]]]*15)[:15])]

# 🤔  
So, how did they lose another 13 chars? Anybody?  

## By the next day ...

An interesting modification of the ```(a,b)[c]``` abbreviation for ```b if c else a``` gave us the following code from [Ali](https://www.kaggle.com/asalhi). This shaved more valuable bytes from the solution.

In [ ]:
p=lambda g:[*zip(*((t:=[*zip(*g)])[:6+2*(t[:4]in[t[4:8],t[8:12]])]*3)[:15])]

# 🎊

Then [Juan Mellado](https://www.kaggle.com/jcmellado) got to the magic **65 bytes** that the double zip was no longer necessary with the latest boolean hack for this data set. Well done Juan and thanks to everybody for contributing.

In [ ]:
p=lambda g:[(r[:6+2*(r[:4]in(r[4:8],r[8:12]))]*3)[:15]for r in g]